# Task 1

In [1]:
import numpy as np
import pandas as pd
import re
import jieba
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support, roc_curve, roc_auc_score
from sklearn.metrics import precision_score,recall_score,f1_score,classification_report
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from functools import reduce
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

d:\program_files\miniconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 使用pandas读入csv数据文件

In [2]:
tiku_question = pd.read_csv('../../camp_dataset/data/tiku_question_sx.csv', encoding='utf-8',usecols = ['que_id', 'content'],low_memory=False)

In [3]:
tiku_question['content'][1]

'<p>问函数$$\\phi \\left( x \\right)=\\sqrt{\\frac{a}{{{x}^{2}}+1}}\\left( a&gt;0 \\right)$$能否成为$$2$$级分裂函数，若能，则求出参数$$a$$的取值范围；若不能请说明理由．</p>'

### 去除非中文字符和空行

In [4]:
def chinese_only(text,key_id,content):
    que_id=[]
    que_content=[]
    line_id = []
    nan_num = 0
    for i in range(text.shape[0]):
        line = text[content][i]
        if not pd.isnull(line):
            line = ''.join(re.findall(r'[\u4e00-\u9fa5]',line))
            if not line=='':
                que_id.append(text[key_id][i])
                que_content.append(' '.join(jieba.cut(line, cut_all=False)))
                line_id.append(i+2)
            else:
                nan_num += 1
        else:
            nan_num += 1
    print(nan_num,'lines deleted(No Chinese Character)!')
    return que_id, que_content, line_id, nan_num

In [5]:
que_id, que_content, line_id, nan_num = chinese_only(tiku_question,'que_id','content')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liang\AppData\Local\Temp\jieba.cache
Loading model cost 0.918 seconds.
Prefix dict has been built succesfully.


1854 lines deleted(No Chinese Character)!


### 计算词袋（tf，tfidf）

In [6]:
def get_bow(text):
    vectorizer = CountVectorizer(min_df=1)
    transformer = TfidfTransformer()
    tf = vectorizer.fit_transform(text)
    tfidf = transformer.fit_transform(tf)
    word = vectorizer.get_feature_names()
    return tf, tfidf, word

In [7]:
tf,tfidf,words = get_bow(que_content)

### 随机选择5道题目

In [8]:
random_q = np.random.randint(0,tf.shape[0],5)

### 计算余弦相似度并输出最相近的3道题目

In [9]:
cosine_tf = cosine_similarity(tf[random_q],tf)
cosine_tfidf = cosine_similarity(tfidf[random_q],tfidf)

In [10]:
for i, r in enumerate(random_q):
    q_ind_rel = []
    line2 = cosine_tfidf[i]
    line2_noself = np.delete(line2, r)
    q_ind = np.argpartition(line2_noself, -3)[-3:]
    for m, k in enumerate(q_ind):
        if k>=r:
            q_ind_rel.append(line_id[k+1])
            q_ind[m] = k+1
        else:
            q_ind_rel.append(line_id[k])
    print('随机抽选的第{}/5题： \n\n题号：{}\n题目内容：{}'.format(i+1,line_id[r],que_content[r]),'\n')
    print('相似度最高的三个题目为：',q_ind_rel,'\n')
    print('{:<7d}{}'.format(q_ind_rel[0],que_content[q_ind[0]]))
    print('{:<7d}{}'.format(q_ind_rel[1],que_content[q_ind[1]]))
    print('{:<7d}{}'.format(q_ind_rel[2],que_content[q_ind[2]]),'\n')

随机抽选的第1/5题： 

题号：56461
题目内容：命题 存在 使得 的 否定 是 

相似度最高的三个题目为： [59087, 6024, 6081] 

59087  命题 使得 的 否定 是
6024   命题 使得 的 否定 是
6081   命题 存在 实数 使得 的 否定 是 

随机抽选的第2/5题： 

题号：80328
题目内容：已知 函数 

相似度最高的三个题目为： [28863, 57999, 9866] 

28863  已知 函数
57999  已知 函数
9866   已知 函数 则 

随机抽选的第3/5题： 

题号：81155
题目内容：若 曲线 有 一切 线 与 直线 垂直 则 切点 可能 为 

相似度最高的三个题目为： [52341, 93076, 29519] 

52341  如果 曲线 的 某 一切 线 与 直线 垂直 求 切点 坐标 与 切线 的 方程
93076  若 曲线 的 某 一切 线 与 直线 垂直 则 切点 坐标 为
29519  若 曲线 有 一切 线 与 直线 垂直 则 切点 可能 为 

随机抽选的第4/5题： 

题号：49963
题目内容：在 中 分别 为 内 角的对边 若且 则 

相似度最高的三个题目为： [65829, 753, 27949] 

65829  在 中 角的对边 分别 为 若且 则 为
753    在 中 角的对边 分别 为 若且 则
27949  设 的 内 角的对边 分别 为 若且 则 

随机抽选的第5/5题： 

题号：3400
题目内容：若求 的 值 

相似度最高的三个题目为： [66633, 81526, 25965] 

66633  若求 的 长
81526  若求 的 值
25965  若求 



# Task 2

### 读入数据文件

In [11]:
knowledge_hierarchy = pd.read_csv('../../camp_dataset/data/knowledge_hierarchy.csv', usecols = ['id', 'name','degree'], encoding = 'utf-8')
question_knowledge_hierarchy = pd.read_csv('../../camp_dataset/data/question_knowledge_hierarchy_sx.csv',  encoding = 'utf-8')

In [12]:
knowledge_hierarchy.head()

,id,name,degree
0,000db1a267624589a55e859ad4b32bfa,运算结果估算,3
1,001c3d25db0a42c29e42d5647299bc5d,囚笼政策,3
2,0025ee71f27b4e3daa9f4a7f164b3962,加减法应用顺口溜,4
3,0026cad84e364586bba7c692f1469a5c,多项式的基本概念,4
4,003203f87c434e7dba19c76e384d1044,夏朝,3


### 寻找类型为“函数与导数”和“三角函数与解三角形”的题目id

In [13]:
knowledge_hierarchy[(knowledge_hierarchy['name']==u'三角函数与解三角形') | (knowledge_hierarchy['name']==u'函数与导数') & (knowledge_hierarchy['degree']==1)]

,id,name,degree
8850,hcwf4avcmp8l53s5iq010pelwlce000d,函数与导数,1
8890,hcwf4avcmp8l53s5iq010pelwlce0035,三角函数与解三角形,1


### 获取满足条件的题目内容

In [14]:
#qkh_fun_id = question_knowledge_hierarchy[question_knowledge_hierarchy['kh_id']=='hcwf4avcmp8l53s5iq010pelwlce000d'].loc[:,['question_id']]
#qkh_tri_id = question_knowledge_hierarchy[question_knowledge_hierarchy['kh_id']=='hcwf4avcmp8l53s5iq010pelwlce0035'].loc[:,['question_id']]
qkh = question_knowledge_hierarchy[(question_knowledge_hierarchy['kh_id']=='hcwf4avcmp8l53s5iq010pelwlce0035') | (question_knowledge_hierarchy['kh_id']=='hcwf4avcmp8l53s5iq010pelwlce000d')].loc[:,['kh_id','question_id']]

In [15]:
tiku_question_new = tiku_question.rename(columns={'que_id':'question_id'})
#question_fun = pd.merge(qkh_fun_id, tiku_question_new, on='question_id')
#question_tri = pd.merge(qkh_tri_id, tiku_question_new, on='question_id')
question_fun_tri = pd.merge(qkh, tiku_question_new, on='question_id')

In [16]:
question_fun_tri.head()

,kh_id,question_id,content
0,hcwf4avcmp8l53s5iq010pelwlce000d,a65e49dd26a0476cb69e32e5f5e511e5,"<p>已知$$f(x)$$为定义在$$(0,+\infty )$$上的连续可导函数，且$$f..."
1,hcwf4avcmp8l53s5iq010pelwlce000d,ff8080814db3e529014df75290f11e7e,<p>已知函数$$f\left( x \right)=\left| \ln x \right...
2,hcwf4avcmp8l53s5iq010pelwlce000d,2965364a3fa14cbf9e3e953d27fb9254,<p>将所有平面向量组成的集合记作$${{R}^{2}}$$，$$f$$是从$${{R}^{...
3,hcwf4avcmp8l53s5iq010pelwlce0035,d568513c37fa4621b1e516ebcaf5bd48,<p>请用&ldquo;五点法&rdquo;画出函数$$f(x)$$在长度为一个周期的闭区间...
4,hcwf4avcmp8l53s5iq010pelwlce000d,a81eb192610c45c98b4480afb6400ee3,<p>求函数$$f(x)$$的极值．</p>


### 计算tfidf值

In [17]:
#que_fun_id, que_fun_content, line_fun_id, nan_fun_num = chinese_only(question_fun,'question_id')
#que_tri_id, que_tri_content, line_tri_id, nan_tri_num = chinese_only(question_tri,'question_id')
que_fun_tri_id, que_fun_tri_content, line_fun_tri_id, nan_fun_tri_num = chinese_only(question_fun_tri,'kh_id','content')
#tf_fun,tfidf_fun,words_fun = get_bow(que_fun_content)
#tf_tri,tfidf_tri,words_tri = get_bow(que_tri_content)
tf_fun_tri,tfidf_fun_tri,words_fun_tri = get_bow(que_fun_tri_content)

1348 lines deleted(No Chinese Character)!


In [18]:
label_lo=[]
for ii in que_fun_tri_id:
    if ii=='hcwf4avcmp8l53s5iq010pelwlce000d':
        label_lo.append('fun')
    else:
        label_lo.append('tri')

### 将数据分为测试集和训练集

In [19]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_fun_tri,label_lo, test_size = 0.5, random_state = 0)

In [20]:
#train_fun_ind, test_fun_ind = train_test_split(range(tfidf_fun.shape[0]), test_size = 0.5, random_state = 0)
#train_tri_ind, test_tri_ind = train_test_split(range(tfidf_tri.shape[0]), test_size = 0.5, random_state = 0)
#train_ind, test_ind = train_test_split(range(tfidf_fun_tri.shape[0]), test_size = 0.5, random_state = 0)

### 效果评价

In [21]:
Logist2 = LogisticRegression()
Logist2.fit(X_train, y_train)
score = Logist2.score(X_test, y_test)
predict2 = Logist2.predict(X_test)
predict_prob2 = Logist2.predict_proba(X_test)[:,1]
precision, recall, f_score, nnn = precision_recall_fscore_support(y_test, predict2)
print('score: ',score,'\n','precision: ',precision,'\n','recall: ',recall,'\n','f_score: ',f_score)
#auc_score = roc_auc_score(y_test, predict2)

score:  0.8522794262134015 
 precision:  [0.87369304 0.78403949] 
 recall:  [0.9280181 0.6607731] 
 f_score:  [0.90003657 0.71714796]


# Task 3

In [22]:
knowledge_hierarchy = pd.read_csv('../../camp_dataset/data/knowledge_hierarchy.csv', encoding='utf-8',usecols = ['id','name'],low_memory=False)
knowledge_hierarchy.head()

,id,name
0,000db1a267624589a55e859ad4b32bfa,运算结果估算
1,001c3d25db0a42c29e42d5647299bc5d,囚笼政策
2,0025ee71f27b4e3daa9f4a7f164b3962,加减法应用顺口溜
3,0026cad84e364586bba7c692f1469a5c,多项式的基本概念
4,003203f87c434e7dba19c76e384d1044,夏朝


In [23]:
tem1, kh, tem2, tem3 = chinese_only(knowledge_hierarchy,'id','name')

1429 lines deleted(No Chinese Character)!


In [24]:
words_all = reduce(lambda x, y: x+' '+y, kh)

In [25]:
#去除重复的字词
words_all_single = list(set(words_all.split()))

### 载入网上已经训练好的模型，来源：https://github.com/Kyubyong/wordvectors

In [26]:
model3 = Word2Vec.load('../../camp_dataset/zh/zh.bin')
dict3 = dict()
for word in words_all_single:
    if word in model3.wv.vocab.keys():
        vec = model3.wv[word]
        dict3[word] = vec

### 随机选取5个词，并判断与其最相近的10个词

In [27]:
n3=0
while True:
    words_random = random.sample(words_all_single, 1)[0]
    if words_random in model3.wv.vocab.keys():
        n3+=1
        print('\n随机抽选的第{}/5个数字： {}'.format(n3,words_random),'\n')
        print('相似度最高的10个词为：\n')
        for i in model3.wv.most_similar(words_random):
            print('{:<10}{:<20}'.format(i[0],i[1]))
        print('\n-------------------')
    if n3==5:
        break


随机抽选的第1/5个数字： 模范 

相似度最高的10个词为：

劳动模范      0.5404252409934998  
荣誉称号      0.49999362230300903 
棒棒        0.49693912267684937 
少儿        0.4604554772377014  
实验学校      0.4591975212097168  
团委        0.4566306471824646  
十佳        0.4556729793548584  
雷锋        0.4536076486110687  
摇篮        0.44403955340385437 
爱国        0.43517807126045227 

-------------------

随机抽选的第2/5个数字： 科学家 

相似度最高的10个词为：

生物学家      0.7366187572479248  
地质学家      0.7248033285140991  
物理学家      0.7125248908996582  
化学家       0.703750729560852   
数学家       0.668906569480896   
心理学家      0.6650621891021729  
学家        0.663875937461853   
学者        0.6625305414199829  
天文学家      0.6580584049224854  
人类学家      0.6567651033401489  

-------------------

随机抽选的第3/5个数字： 灵魂 

相似度最高的10个词为：

肉体        0.7606858611106873  
心灵        0.7533289194107056  
梦境        0.6931494474411011  
复活        0.6864750385284424  
永恒        0.6830259561538696  
救赎        0.6646701097488403  
恶魔        0.6644892692565918  
邪恶        0.662882

d:\program_files\miniconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Task 4

In [28]:
model4 = Word2Vec([words_all.split()],min_count=1,size=200)

In [29]:
dict4 = dict()
for word in words_all_single:
    if word in model4.wv.vocab.keys():
        vec = model4.wv[word]
        dict4[word] = vec
n4 = 0
while True:
    words_random = random.sample(words_all_single,1)[0]
    if words_random in model4.wv.vocab.keys():
        n4 += 1
        print('随机抽选的第{}/5个数字： {}'.format(n4,words_random),'\n')
        print('相似度最高的10个词为：\n')
        for i in model4.wv.most_similar(words_random):
            print('{:<10}{}'.format(i[0],i[1]))
        print('\n---------------')
    if n4 == 5:
        break

随机抽选的第1/5个数字： 数 

相似度最高的10个词为：

的         0.9919224977493286
和         0.990704357624054
与         0.9901132583618164
应用        0.9894493818283081
中         0.989235520362854
实验        0.9878840446472168
解         0.9874417781829834
问题        0.9874029159545898
定义        0.9869540929794312
及其        0.9867161512374878

---------------
随机抽选的第2/5个数字： 三视图 

相似度最高的10个词为：

等         0.8120992183685303
一元二次方程    0.8108184337615967
特点        0.8106659650802612
与         0.8067290782928467
细胞        0.8064182996749878
及其        0.8062081336975098
整数        0.8061712980270386
关系        0.8060632348060608
的         0.8053901195526123
探究        0.8047338724136353

---------------
随机抽选的第3/5个数字： 合数 

相似度最高的10个词为：

文艺复兴      0.3154507875442505
爱琴        0.3078083395957947
小方块       0.2833520472049713
直接        0.2833397090435028
代换        0.2800452709197998
英国        0.2796751856803894
制备        0.2789420485496521
结合        0.27851468324661255
公共        0.2771608233451843
掌握        0.276101738214492

d:\program_files\miniconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Task 5

In [30]:
que_fun_tri_content[0:5]

['已知 为 定义 在 上 的 连续 可导 函数 且 则 不等式 的 解集 为',
 '已知 函数 则 方程 实根 的 个数 为',
 '将 所有 平面 向量 组成 的 集合 记 作 是从 到 的 映射 记作 或 其中 都 是 实数 定义 映射 的 模为 在 的 条件 下 的 最大值 记做 若 存在 非零 向量 及 实数 使得 则 称为 的 一个 特征值',
 '请 用 五点 法画 出 函数 在 长度 为 一个 周期 的 闭 区间 上 的 简图 先 在 所 给 的 表格 中填上 所 需 的 数值 再 画图',
 '求函数 的 极值']

In [31]:
def data_tage(cont):
    data_taged = []
    for i, doc in enumerate(cont):
        docu = TaggedDocument(doc.split(),[i])
        data_taged.append(docu)
    return data_taged

In [32]:
data_taged = data_tage(que_fun_tri_content)

In [33]:
if 'model5' in vars():
    del model5
model5 = Doc2Vec(data_taged,min_count=1,vector_size=500)

### 计算每个题目的向量

In [34]:
data_vecs = []
for i in range(len(data_taged)):
    data_vecs.append(model5.infer_vector(data_taged[i].words))

### 划分训练集和测试集并进行Logist回归

In [35]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(data_vecs,label_lo, test_size = 0.5, random_state = 0)

In [36]:
Logist5 = LogisticRegression()
Logist5.fit(X_train5, y_train5)
score5 = Logist5.score(X_test5, y_test5)
predict5 = Logist5.predict(X_test5)
predict_prob5 = Logist5.predict_proba(X_test5)[:,1]
precision5, recall5, f_score5, nnn5 = precision_recall_fscore_support(y_test5, predict5)
print('score: ',score5,'\n','precision: ',precision5,'\n','recall: ',recall5,'\n','f_score: ',f_score5)
#auc_score = roc_auc_score(y_test5, predict5)

score:  0.7220966791118097 
 precision:  [0.72600729 0.59333333] 
 recall:  [0.98327278 0.06170914] 
 f_score:  [0.83527939 0.11179149]
